In [3]:
import pandas as pd
import urllib3 as urllib
import json
from fuzzywuzzy import process
import IPython.display

In [30]:
# state_name = 'OH'
# max_date = '2020-11-04'
# state_file = '/home/steve/Downloads/OH-2020-11-01 - normalized-state.tsv'
# cms_file = '/home/steve/Downloads/OH-2020-11-01 - normalized-cms.tsv'

state_name = 'ID'
max_date = '2020-11-05'
state_file = '/home/steve/Downloads/ID-2020-11-01 - normalized-state (1).tsv'
cms_file = '/home/steve/Downloads/ID-2020-11-01 - normalized-cms (1).tsv'

In [31]:
state_df = pd.read_csv(state_file, sep='\t')
cms_df = pd.read_csv(cms_file, sep='\t')
state_df = state_df.drop_duplicates(subset=['County', 'Facility Name']).copy()

In [32]:
state_df.columns

Index(['Facility Name', 'Facility Name Normalized', 'Changed', 'VLOOKUP',
       'Date Collected', 'State', 'County', 'City', 'State Facility Type',
       'CTP Facility Categorization', 'State/Fed Regulated',
       'State Facility ID', 'CMS Facility ID', 'Date outbreak opened',
       'Date outreak closed', 'Outbreak Status', 'Resident Census',
       'Resident Positives', 'Resident Probable Positives', 'Resident Deaths',
       'Resident Probable Deaths', 'Staff Positive',
       'Staff Probable Positives', 'Staff Deaths', 'Staff Probable Deaths',
       'Resident/Staff Positives', 'Resident/Staff Probable Positives',
       'Resident/Staff Deaths', 'Resident/Staff Probable Deaths',
       'Resident Positives.1', 'Resident Probable Positives.1',
       'Resident Deaths.1', 'Resident Probable Deaths.1', 'Staff Positive.1',
       'Staff Probable Positives.1', 'Staff Deaths.1',
       'Staff Probable Deaths.1', 'Resident/Staff Positives.1',
       'Resident/Staff Probable Positives.1'

In [33]:
state_df.sort_values(by=['County', 'Facility Name'], inplace=True)
state_df.head(5)

,Facility Name,Facility Name Normalized,Changed,VLOOKUP,Date Collected,State,County,City,State Facility Type,CTP Facility Categorization,...,Staff Probable Positives.1,Staff Deaths.1,Staff Probable Deaths.1,Resident/Staff Positives.1,Resident/Staff Probable Positives.1,Resident/Staff Deaths.1,Resident/Staff Probable Deaths.1,Residents Tested,Staff Tested,Personal Protective Equipment
0,AarenBrooke Place,AARENBROOKE PLACE,NaN,NaN,20201105,ID,ADA,BOISE,NaN,Uncategorized LTC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Arbor Valley of Cascadia,ARBOR VALLEY OF CASCADIA,NaN,ARBOR VALLEY OF CASCADIA,20201105,ID,ADA,BOISE,NaN,Uncategorized LTC,...,NaN,NaN,NaN,21.0,NaN,0.0,NaN,NaN,NaN,NaN
7,Arbor Village at Hillcrest,ARBOR VILLAGE AT HILLCREST,NaN,NaN,20201105,ID,ADA,BOISE,NaN,Uncategorized LTC,...,NaN,NaN,NaN,17.0,NaN,0.0,NaN,NaN,NaN,NaN
11,Ashley Manor – Cloverdale,ASHLEY MANOR – CLOVERDALE,NaN,NaN,20201105,ID,ADA,BOISE,NaN,Uncategorized LTC,...,NaN,NaN,NaN,4.0,NaN,0.0,NaN,NaN,NaN,NaN
12,Ashley Manor – Eagle House #2,ASHLEY MANOR – EAGLE HOUSE #2,NaN,NaN,20201105,ID,ADA,EAGLE,NaN,Uncategorized LTC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
display(state_df.columns)
display(cms_df.columns)

Index(['Facility Name', 'Facility Name Normalized', 'Changed', 'VLOOKUP',
       'Date Collected', 'State', 'County', 'City', 'State Facility Type',
       'CTP Facility Categorization', 'State/Fed Regulated',
       'State Facility ID', 'CMS Facility ID', 'Date outbreak opened',
       'Date outreak closed', 'Outbreak Status', 'Resident Census',
       'Resident Positives', 'Resident Probable Positives', 'Resident Deaths',
       'Resident Probable Deaths', 'Staff Positive',
       'Staff Probable Positives', 'Staff Deaths', 'Staff Probable Deaths',
       'Resident/Staff Positives', 'Resident/Staff Probable Positives',
       'Resident/Staff Deaths', 'Resident/Staff Probable Deaths',
       'Resident Positives.1', 'Resident Probable Positives.1',
       'Resident Deaths.1', 'Resident Probable Deaths.1', 'Staff Positive.1',
       'Staff Probable Positives.1', 'Staff Deaths.1',
       'Staff Probable Deaths.1', 'Resident/Staff Positives.1',
       'Resident/Staff Probable Positives.1'

Index(['Provider Name', 'Federal Provider Number', 'Provider Address',
       'Provider City', 'County', 'Residents Total Confirmed COVID-19',
       'Residents Total Suspected COVID-19', 'Residents Total All Deaths',
       'Residents Total COVID-19 Deaths', 'Total Number of Occupied Beds',
       'Staff Total Confirmed COVID-19', 'Staff Total Suspected COVID-19',
       'Staff Total COVID-19 Deaths'],
      dtype='object')

In [35]:
cols=['County', 'Facility Name Normalized', 'VLOOKUP', 'VLOOKUP2', 'Change Score']
state_df['VLOOKUP2'] = '' # For Spreadsheet
state_df['Change Score'] = '' # For Spreadsheet
for county in list(cms_df['County'].unique()):
    #print(county)
    # Get lists of Facilities for the counties
    df_state = state_df[state_df['County'] == county]
    df_cms = cms_df[cms_df['County'] == county]
    state_facs = list(df_state['Facility Name Normalized'])
    cms_facs = list(df_cms['Provider Name'])
    for str2Match in state_facs:
        if (str2Match in cms_facs):
            state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'VLOOKUP2'] = str2Match            
            state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'Change Score'] = 100
            cms_facs.remove(str2Match)
        else:
            try:
                Ratios = process.extract(str2Match,cms_facs, limit=2)
                #print(Ratios)
                state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'VLOOKUP2'] = Ratios[0][0]            
                state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'Change Score'] = Ratios[0][1]
            except:
                pass

state_df.drop_duplicates(subset=['County', 'Facility Name Normalized']).copy()['Change Score'].value_counts()

100    73
       26
86     22
42     11
41      9
43      8
38      8
45      7
39      7
40      6
36      5
44      5
32      4
35      4
47      4
46      4
37      3
48      3
52      3
33      2
30      2
90      2
50      2
65      2
53      1
34      1
49      1
95      1
25      1
76      1
75      1
54      1
Name: Change Score, dtype: int64

In [36]:
state_df['VlookupMatch'] = state_df['VLOOKUP'] == state_df['VLOOKUP2']
state_df[cols].head(10)
state_df['VlookupMatch'].head(10)

0     False
6      True
7     False
11    False
12    False
13    False
17     True
18    False
19    False
22    False
Name: VlookupMatch, dtype: bool

In [37]:
# MI_cms_data[MI_cms_data['County'] == 'ALLEGAN']
# MI_cms_data['County'].unique()
# MI_state_data['County'].unique()

In [38]:
#display(MI_state_data['Change Score'].value_counts())
state_df.to_csv('./Reporting_data/' + state_name + '_State_to_CMS_Data_' + str(max_date)[0:10] + '.csv', index=False)
state_df[cols].head(20)

,County,Facility Name Normalized,VLOOKUP,VLOOKUP2,Change Score
0,ADA,AARENBROOKE PLACE,NaN,AVAMERE TRANSITIONAL CARE,43
6,ADA,ARBOR VALLEY OF CASCADIA,ARBOR VALLEY OF CASCADIA,ARBOR VALLEY OF CASCADIA,100
7,ADA,ARBOR VILLAGE AT HILLCREST,NaN,GOOD SAMARITAN SOCIETY – BOISE VILLAGE,52
11,ADA,ASHLEY MANOR – CLOVERDALE,NaN,LIFE CARE OF TREASURE VALLEY,45
12,ADA,ASHLEY MANOR – EAGLE HOUSE #2,NaN,LIFE CARE OF BOISE,43
13,ADA,ASHLEY MANOR – HYDE PARK,NaN,SHAW MOUNTAIN OF CASCADIA,41
17,ADA,ASPEN TRANSITIONAL REHABILITATION,ASPEN TRANSITIONAL REHABILITATION,ASPEN TRANSITIONAL REHABILITATION,100
18,ADA,ASPEN VALLEY SENIOR LIVING,NaN,LIFE CARE OF TREASURE VALLEY,53
19,ADA,ASPIRE – FIELDSTONE,NaN,CASCADIA OF BOISE,44
22,ADA,AUTUMN COVE,NaN,RIVERVIEW REHABILITATION,47


In [39]:
len(state_df['Facility Name'].unique())

229

In [40]:
state_df['county-name'] = state_df['County'] + state_df['Facility Name']
state_df_unique = state_df.drop_duplicates(subset=['County', 'Facility Name']).copy()

In [41]:
print(len(state_df['county-name'].unique()))
len(state_df_unique)
#len(state_df)

230


230

In [42]:
cms_df['county-name'] = cms_df['County'] + state_df['Facility Name']
len(cms_df['county-name'].unique())

82

In [43]:
cms_df.columns

Index(['Provider Name', 'Federal Provider Number', 'Provider Address',
       'Provider City', 'County', 'Residents Total Confirmed COVID-19',
       'Residents Total Suspected COVID-19', 'Residents Total All Deaths',
       'Residents Total COVID-19 Deaths', 'Total Number of Occupied Beds',
       'Staff Total Confirmed COVID-19', 'Staff Total Suspected COVID-19',
       'Staff Total COVID-19 Deaths', 'county-name'],
      dtype='object')

# Play Area

In [ ]:
if ('WEISER CARE OF CASCADIA' in cms_facs):
    print("In List")
    cms_facs.remove('WEISER CARE OF CASCADIA')

In [ ]:
cms_facs.append('AARENBROOKE PLACE')

In [ ]:
cms_facs

In [ ]:
cols=['County', 'Facility Name Normalized', 'VLOOKUP', 'VLOOKUP2', 'Change Score']
state_df['VLOOKUP2'] = '' # For Spreadsheet
state_df['Change Score'] = '' # For Spreadsheet
for county in list(cms_df['County'].unique()):
    #print(county)
    # Get lists of Facilities for the counties
    df_state = state_df[state_df['County'] == county]
    df_cms = cms_df[cms_df['County'] == county]
    state_facs = list(df_state['Facility Name Normalized'])
    cms_facs = list(df_cms['Provider Name'])
    for str2Match in state_facs:
        try:
            Ratios = process.extract(str2Match,cms_facs, limit=2)
            #print(Ratios)
            state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'VLOOKUP2'] = Ratios[0][0]            
            state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'Change Score'] = Ratios[0][1]
        except:
            pass

state_df['Change Score'].value_counts()
# str2Match = cms_facs[0]
# strOptions = state_facs

# # Pass 1 Remove Exact Matches
# for str2Match in cms_facs:
#     Ratios = process.extract(str2Match,strOptions, limit=3)
#     print(str2Match, Ratios)

In [ ]:
county='STARK'


cols=['County', 'Facility Name Normalized', 'VLOOKUP', 'VLOOKUP2', 'Change Score']
state_df['VLOOKUP2'] = '' # For Spreadsheet
state_df['Change Score'] = '' # For Spreadsheet
#for county in list(cms_df['County'].unique()):
#print(county)
# Get lists of Facilities for the counties
df_state = state_df[state_df['County'] == county]
df_cms = cms_df[cms_df['County'] == county]
state_facs = list(df_state['Facility Name Normalized'])
cms_facs = list(df_cms['Provider Name'])
for str2Match in state_facs:
    if (str2Match in cms_facs):
        state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'VLOOKUP2'] = str2Match            
        state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'Change Score'] = 100
        cms_facs.remove(str2Match)

for str2Match in state_facs:
    try:
        Ratios = process.extract(str2Match,cms_facs, limit=2)
        #print(Ratios)
        state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'VLOOKUP2'] = Ratios[0][0]            
        state_df.loc[(state_df['County'] == county.upper()) & (state_df['Facility Name Normalized'] == str2Match), 'Change Score'] = Ratios[0][1]
    except:
        pass

state_df.drop_duplicates(subset=['County', 'Facility Name Normalized']).copy()['Change Score'].value_counts()
print(cms_facs)

In [ ]:
df_state = state_df[state_df['County'] == county]
df_cms = cms_df[cms_df['County'] == county]
state_facs = list(df_state['Facility Name Normalized'])
cms_facs = list(df_cms['Provider Name'])-
df_cms = cms_df[cms_df['County'] == county]
df_cms

In [ ]:
df_cms